In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras import applications
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB7, preprocess_input
from tensorflow.keras import Input
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
from pickle import dump, load
from PIL import Image
from tqdm import tqdm
tqdm().pandas()
from google.cloud import storage
import os

2024-03-11 23:10:36.662094: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 23:10:36.668201: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 23:10:36.760587: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 23:10:36.760642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 23:10:36.761814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [ ]:
def download_bucket_with_transfer_manager(
    bucket_name, destination_directory="", workers=8, max_results=1000
):
    from google.cloud.storage import Client, transfer_manager

    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    blob_names = [blob.name for blob in bucket.list_blobs(max_results=max_results)]

    results = transfer_manager.download_many_to_path(
        bucket, blob_names, destination_directory=destination_directory, max_workers=workers
    )

    for name, result in zip(blob_names, results):
        # The results list is either `None` or an exception for each blob in
        # the input list, in order.

        if isinstance(result, Exception):
            print("Failed to download {} due to exception: {}".format(name, result))
        else:
            print("Downloaded {} to {}.".format(name, destination_directory + name))

In [ ]:
#download_bucket_with_transfer_manager(
#    'covid19_lewagon', destination_directory="/home/user/code/gulfairus/Covid19/raw_data/cloud", workers=8, max_results=25500
#)

In [10]:
import os
main_dir = "/home/user/code/gulfairus/Covid19/raw_data/cloud/COVID-19_Radiography_Dataset"
covid = os.path.join(main_dir, "COVID")
normal = os.path.join(main_dir, "Normal")
pneumonia = os.path.join(main_dir, "Viral Pneumonia")
opacity = os.path.join(main_dir, "Lung_Opacity")
covid_names = os.listdir(covid)
normal_names = os.listdir(normal)
pneumonia_names = os.listdir(pneumonia)
opacity_names = os.listdir(opacity)
print(len(covid_names))
print(len(normal_names))
print(len(pneumonia_names))
print(len(opacity_names))
print(len(covid_names)+len(normal_names)+len(pneumonia_names)+len(opacity_names))


3616
10192
1345
6012
21165


In [8]:
covid_names = os.listdir(covid)
normal_names = os.listdir(normal)
pneumonia_names = os.listdir(pneumonia)
opacity_names = os.listdir(opacity)

In [9]:
print(len(covid_names))
print(len(normal_names))
print(len(pneumonia_names))
print(len(opacity_names))
print(len(covid_names)+len(normal_names)+len(pneumonia_names)+len(opacity_names))

3616
10192
1345
6012
21165
